# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 10 2023 9:38AM,257123,16,SSF_C2W7,Sartorius Stedim Filters Inc.,Released
1,Feb 10 2023 9:34AM,257066,15,VT80183914,"Virtus Pharmaceuticals, LLC",Released
2,Feb 10 2023 9:33AM,257005,15,9133524,"Mizner Bioscience, LLC",Released
3,Feb 10 2023 9:33AM,257005,15,9133525,"Mizner Bioscience, LLC",Released
4,Feb 10 2023 9:33AM,257005,15,9133526,"Mizner Bioscience, LLC",Released
5,Feb 10 2023 9:31AM,257065,15,PNC425602A,"Person & Covey, Inc.",Released
6,Feb 10 2023 9:31AM,257065,15,PNC425613A,"Person & Covey, Inc.",Released
7,Feb 10 2023 9:31AM,257065,15,PNC425615A,"Person & Covey, Inc.",Released
8,Feb 10 2023 9:31AM,257065,15,PNC425617A,"Person & Covey, Inc.",Released
9,Feb 10 2023 9:31AM,257065,15,PNC425623A,"Person & Covey, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
41,257119,Released,12
42,257120,Released,1
43,257121,Released,1
44,257122,Released,1
45,257123,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
257119,NaN,12.0
257120,NaN,1.0
257121,NaN,1.0
257122,NaN,1.0
257123,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
256949,0.0,47.0
257004,0.0,1.0
257005,0.0,3.0
257024,0.0,6.0
257027,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
256949,0,47
257004,0,1
257005,0,3
257024,0,6
257027,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,256949,0,47
1,257004,0,1
2,257005,0,3
3,257024,0,6
4,257027,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,256949,,47
1,257004,,1
2,257005,,3
3,257024,,6
4,257027,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 10 2023 9:38AM,257123,16,Sartorius Stedim Filters Inc.
1,Feb 10 2023 9:34AM,257066,15,"Virtus Pharmaceuticals, LLC"
2,Feb 10 2023 9:33AM,257005,15,"Mizner Bioscience, LLC"
5,Feb 10 2023 9:31AM,257065,15,"Person & Covey, Inc."
17,Feb 10 2023 9:30AM,257122,10,Eywa Pharma Inc.
18,Feb 10 2023 9:26AM,256949,15,"Person & Covey, Inc."
65,Feb 10 2023 9:24AM,257092,15,"Alliance Pharma, Inc."
66,Feb 10 2023 9:24AM,257064,15,"Alliance Pharma, Inc."
68,Feb 10 2023 9:22AM,257121,16,Sartorius Stedim Filters Inc.
69,Feb 10 2023 9:15AM,257120,10,Eywa Pharma Inc.


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Feb 10 2023 9:38AM,257123,16,Sartorius Stedim Filters Inc.,,1
1,Feb 10 2023 9:34AM,257066,15,"Virtus Pharmaceuticals, LLC",,1
2,Feb 10 2023 9:33AM,257005,15,"Mizner Bioscience, LLC",,3
3,Feb 10 2023 9:31AM,257065,15,"Person & Covey, Inc.",,12
4,Feb 10 2023 9:30AM,257122,10,Eywa Pharma Inc.,,1
5,Feb 10 2023 9:26AM,256949,15,"Person & Covey, Inc.",,47
6,Feb 10 2023 9:24AM,257092,15,"Alliance Pharma, Inc.",,1
7,Feb 10 2023 9:24AM,257064,15,"Alliance Pharma, Inc.",,2
8,Feb 10 2023 9:22AM,257121,16,Sartorius Stedim Filters Inc.,,1
9,Feb 10 2023 9:15AM,257120,10,Eywa Pharma Inc.,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Feb 10 2023 9:38AM,257123,16,Sartorius Stedim Filters Inc.,1,
1,Feb 10 2023 9:34AM,257066,15,"Virtus Pharmaceuticals, LLC",1,
2,Feb 10 2023 9:33AM,257005,15,"Mizner Bioscience, LLC",3,
3,Feb 10 2023 9:31AM,257065,15,"Person & Covey, Inc.",12,
4,Feb 10 2023 9:30AM,257122,10,Eywa Pharma Inc.,1,
5,Feb 10 2023 9:26AM,256949,15,"Person & Covey, Inc.",47,
6,Feb 10 2023 9:24AM,257092,15,"Alliance Pharma, Inc.",1,
7,Feb 10 2023 9:24AM,257064,15,"Alliance Pharma, Inc.",2,
8,Feb 10 2023 9:22AM,257121,16,Sartorius Stedim Filters Inc.,1,
9,Feb 10 2023 9:15AM,257120,10,Eywa Pharma Inc.,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Feb 10 2023 9:38AM,257123,16,Sartorius Stedim Filters Inc.,1,
1,Feb 10 2023 9:34AM,257066,15,"Virtus Pharmaceuticals, LLC",1,
2,Feb 10 2023 9:33AM,257005,15,"Mizner Bioscience, LLC",3,
3,Feb 10 2023 9:31AM,257065,15,"Person & Covey, Inc.",12,
4,Feb 10 2023 9:30AM,257122,10,Eywa Pharma Inc.,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Feb 10 2023 9:38AM,257123,16,Sartorius Stedim Filters Inc.,1.0,NaN
1,Feb 10 2023 9:34AM,257066,15,"Virtus Pharmaceuticals, LLC",1.0,NaN
2,Feb 10 2023 9:33AM,257005,15,"Mizner Bioscience, LLC",3.0,NaN
3,Feb 10 2023 9:31AM,257065,15,"Person & Covey, Inc.",12.0,NaN
4,Feb 10 2023 9:30AM,257122,10,Eywa Pharma Inc.,1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Feb 10 2023 9:38AM,257123,16,Sartorius Stedim Filters Inc.,1.0,0.0
1,Feb 10 2023 9:34AM,257066,15,"Virtus Pharmaceuticals, LLC",1.0,0.0
2,Feb 10 2023 9:33AM,257005,15,"Mizner Bioscience, LLC",3.0,0.0
3,Feb 10 2023 9:31AM,257065,15,"Person & Covey, Inc.",12.0,0.0
4,Feb 10 2023 9:30AM,257122,10,Eywa Pharma Inc.,1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,5912864,157.0,7.0
12,1028365,25.0,0.0
15,1799245,67.0,0.0
16,771351,3.0,0.0
18,257106,1.0,0.0
19,1542562,26.0,0.0
20,257027,0.0,1.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,5912864,157.0,7.0
1,12,1028365,25.0,0.0
2,15,1799245,67.0,0.0
3,16,771351,3.0,0.0
4,18,257106,1.0,0.0
5,19,1542562,26.0,0.0
6,20,257027,0.0,1.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,157.0,7.0
1,12,25.0,0.0
2,15,67.0,0.0
3,16,3.0,0.0
4,18,1.0,0.0
5,19,26.0,0.0
6,20,0.0,1.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,157.0
1,12,Released,25.0
2,15,Released,67.0
3,16,Released,3.0
4,18,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,18,19,20
Status,,,,,,,
Executing,7.0,0.0,0.0,0.0,0.0,0.0,1.0
Released,157.0,25.0,67.0,3.0,1.0,26.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,18,19,20
0,Executing,7.0,0.0,0.0,0.0,0.0,0.0,1.0
1,Released,157.0,25.0,67.0,3.0,1.0,26.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,18,19,20
0,Executing,7.0,0.0,0.0,0.0,0.0,0.0,1.0
1,Released,157.0,25.0,67.0,3.0,1.0,26.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()